# Genie API Test - Get & Update Space

Test notebook for validating Genie Space connectivity and updating space configuration.

## API Endpoints
1. `GET /api/2.0/genie/spaces/{space_id}?include_serialized_space=true` - Get space details
2. `PATCH /api/2.0/genie/spaces/{space_id}` - Update space with serialized_space

## Parameters
- **genie_space_id** (required): The ID of the Genie space
- **source_root**: Folder inside volume (e.g., test) for serialized_space.json path
- **catalog**: Catalog name (e.g., aira_test)

## Workflow
1. Get current space configuration
2. Load serialized_space.json from Volumes (generated by job_cdm_export_genie_assets)
3. Update the Genie space with the new configuration

## References
- [Databricks Genie API - getSpace](https://docs.databricks.com/api/workspace/genie/getspace)
- [Databricks Genie API - updateSpace](https://docs.databricks.com/api/workspace/genie/updatespace)


In [ ]:
# Cell 1: Configuration
import json
import os

# Widgets for configuration
dbutils.widgets.text("genie_space_id", "01f0da24c4fd18a6aae15be618310c9b", "Genie Space ID")
dbutils.widgets.text("source_root", "test", "Source Root")
dbutils.widgets.text("catalog", "aira_test", "Catalog")

GENIE_SPACE_ID = dbutils.widgets.get("genie_space_id")
source_root = dbutils.widgets.get("source_root")
catalog = dbutils.widgets.get("catalog")

if not GENIE_SPACE_ID:
    print("⚠️ Please provide a Genie Space ID")
    print("   Find it in: Databricks Workspace > Genie > [Your Space] > Copy from URL")
    dbutils.notebook.exit("Missing genie_space_id parameter")

# Build path to serialized_space.json
source_volume = "clinical_data_standards"
bronze_schema = "bronze_md"
serialized_space_path = f"/Volumes/{catalog}/{bronze_schema}/{source_volume}/{source_root}/exports/genie/serialized_space.json"

print(f"="*60)
print(f"GENIE API TEST CONFIGURATION")
print(f"="*60)
print(f"Genie Space ID: {GENIE_SPACE_ID}")
print(f"Catalog: {catalog}")
print(f"Source Root: {source_root}")
print(f"Serialized Space Path: {serialized_space_path}")
print(f"="*60)


In [ ]:
# Cell 2: Get Genie Space API Test
# API: GET /api/2.0/genie/spaces/{space_id}?include_serialized_space=true
# Reference: https://docs.databricks.com/api/workspace/genie/getspace

import requests

# Get workspace URL and token from the current context
workspace_url = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiUrl().get()
token = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()

headers = {
    "Authorization": f"Bearer {token}",
    "Content-Type": "application/json"
}

print("="*60)
print("GET GENIE SPACE")
print("="*60)

# Call getSpace API via REST
api_url = f"{workspace_url}/api/2.0/genie/spaces/{GENIE_SPACE_ID}"

params = {
    "include_serialized_space": "true"
}

response = requests.get(api_url, headers=headers, params=params)
space = response.json()

# Parse serialized_space if it's a string (so it prints nicely)
if isinstance(space.get('serialized_space'), str):
    space['serialized_space'] = json.loads(space['serialized_space'])

print(f"✓ Space ID: {space.get('space_id')}")
print(f"✓ Title: {space.get('title')}")
print(f"✓ Description: {space.get('description', '')[:100]}...")

# Print the complete response (beautified)
print(f"\n📋 Full Response:")
print(json.dumps(space, indent=2))


In [ ]:
# Cell 3: Update Genie Space API
# API: PATCH /api/2.0/genie/spaces/{space_id}
# Reference: https://docs.databricks.com/api/workspace/genie/updatespace

print("\n" + "="*60)
print("UPDATE GENIE SPACE")
print("="*60)

# Check if serialized_space.json exists
if not os.path.exists(serialized_space_path):
    print(f"❌ File not found: {serialized_space_path}")
    print("   Run job_cdm_export_genie_assets first to generate the file.")
    dbutils.notebook.exit(json.dumps({"success": False, "error": "serialized_space.json not found"}))

# Read the serialized_space.json
with open(serialized_space_path, 'r') as f:
    serialized_space_content = json.load(f)

print(f"✓ Loaded serialized_space.json")
print(f"  Tables: {len(serialized_space_content.get('data_sources', {}).get('tables', []))}")
print(f"  Sample Questions: {len(serialized_space_content.get('config', {}).get('sample_questions', []))}")
print(f"  Example SQLs: {len(serialized_space_content.get('instructions', {}).get('example_question_sqls', []))}")

# Convert to string for API (serialized_space must be a JSON string in the request)
serialized_space_str = json.dumps(serialized_space_content)

# Call updateSpace API via REST
api_url = f"{workspace_url}/api/2.0/genie/spaces/{GENIE_SPACE_ID}"

update_payload = {
    "serialized_space": serialized_space_str
}

print(f"\n📤 Calling PATCH {api_url}")

response = requests.patch(api_url, headers=headers, json=update_payload)

if response.status_code == 200:
    result = response.json()
    print(f"\n✅ SUCCESS: Genie Space updated")
    print(f"   Space ID: {result.get('space_id')}")
    print(f"   Title: {result.get('title')}")
    
    update_result = {
        "success": True,
        "space_id": result.get('space_id'),
        "title": result.get('title'),
        "serialized_space_path": serialized_space_path
    }
else:
    error_body = response.text
    print(f"\n❌ FAILED ({response.status_code}):")
    try:
        error_json = json.loads(error_body)
        print(json.dumps(error_json, indent=2))
    except:
        print(error_body)
    
    update_result = {
        "success": False,
        "status_code": response.status_code,
        "error": error_body
    }

print(f"\n{'='*60}")
print("UPDATE COMPLETE")
print(f"{'='*60}")

dbutils.notebook.exit(json.dumps(update_result, indent=2))
